# Improve ChatGPT with Knowledge Graphs


In [1]:
# pip install -q openai langchain

In [18]:
# %reset -f


In [2]:
# pip install python-dotenv

In [2]:
import os
import openai
from string import Template
import json
from neo4j import GraphDatabase
import glob
from timeit import default_timer as timer
from dotenv import load_dotenv
from time import sleep
from langchain.text_splitter import TokenTextSplitter
from langchain.schema import Document


In [3]:
load_dotenv()

True

In [4]:
import os
import openai

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = os.getenv("api_version")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("api_key_azure")

# os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_KEY")
# openai.api_key = os.getenv("api_key_azure")
# openai.api_version = os.getenv("api_version")
openai_deployment = "sdgi-gpt-35-turbo-16k" 
# print(os.getenv("api_key_azure"))
# print("=====")
# print(os.getenv("api_version"))
# completion = openai.ChatCompletion.create(model="gpt-3.5-turbo",
#                                           temperature=0,
#                                           messages=[{"role": "user",
#                                                      "content": question}])
# print(completion["choices"][0]["message"]["content"])

# completion = openai.chat.completions.create(
#                     model=openai_deployment,
#                     max_tokens=15000,
#                     temperature=0,
#                     messages=[
#                         {"role": "user", "content": question}
#                     ]
#                 )
# nlp_results = completion.choices[0].message.content
# print(nlp_results)

# print(os.environ['OPENAI_API_BASE'])

In [5]:
# from langchain.llms import OpenAI
from langchain.indexes import GraphIndexCreator
from langchain.chains import GraphQAChain
from langchain.prompts import PromptTemplate
# from langchain.llms import AzureChatOpenAI
from langchain.chat_models import AzureChatOpenAI
import networkx as nx
import concurrent.futures
from tqdm import tqdm  # Import tqdm for the progress bar


def custom_sentence_tokenize(text, max_words=250):
    words = text.split()
    sentences = []
    current_sentence = []

    for word in words:
        current_sentence.append(word)
        if len(current_sentence) >= max_words:
            sentences.append(' '.join(current_sentence))
            current_sentence = []

    if current_sentence:
        sentences.append(' '.join(current_sentence))

    return sentences

 
def createGML(text, filename):
     
    sentences = custom_sentence_tokenize(text)
    print(f"********* {len(sentences)}")
    sleep(8)
    try:
        for idx, sentence in enumerate(sentences):
            # Create a graph for each sentence
            prompt_extractor= "You are a networked intelligence helping a human track knowledge triples about all relevant people, things, concepts, names,places, Dates and Times, Numbers, Organizations, Products and Brands, Events, Roles and Positions, Keywords and Topics, Email Addresses and URLs, References to External Entities, Emotional Tone, Quantities and Units, Codes and Identifiers, Languages, Social Media Handles, Currencies etc. and integrating them with your knowledge stored within your weights as well as that stored in a knowledge graph. Extract all of the knowledge triples from the text. A knowledge triple is a clause that contains a subject, a predicate, and an object. The subject is the entity being described, the predicate is the property of the subject that is being described, and the object is the value of the property.\n\nEXAMPLE\nIt's a state in the US. It's also the number 1 producer of gold in the US.\n\nOutput: (Nevada, is a, state)<|>(Nevada, is in, US)<|>(Nevada, is the number 1 producer of, gold)\nEND OF EXAMPLE\n\nEXAMPLE\nI'm going to the store.\n\nOutput: NONE\nEND OF EXAMPLE\n\nEXAMPLE\nOh huh. I know Descartes likes to drive antique scooters and play the mandolin.\nOutput: (Descartes, likes to drive, antique scooters)<|>(Descartes, plays, mandolin)\nEND OF EXAMPLE\n\nEXAMPLE\n{text}Output:"
            index_creator = GraphIndexCreator(llm=AzureChatOpenAI(deployment_name=openai_deployment, temperature=0))
            graph = index_creator.from_text(sentence, prompt=PromptTemplate(input_variables=['text'], template=prompt_extractor))
            # print(f" filename=== {filename} and idx=== {idx} ")
            # Write each graph to a GML file with a unique filename
            filenamex = f"Models/GML/{filename}_{idx + 1}.gml"  # Unique filename based on index
            graph.write_to_gml(filenamex)
    except Exception as e:
        print(f"Error creating GML file {e}")    


def process_file(file):
    try:
        with open(file, "r") as f:
            text = f.read().rstrip()
            filename = file.split("/")[-1].replace(".txt", "")
            createGML(text, filename)
    except Exception as e:
        print(f"Error processing {file}: {e}")

# def process_files_concurrently(files):
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         executor.map(process_file, files)


def process_files_concurrently(files):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Use tqdm to create a progress bar
        with tqdm(total=len(files)) as pbar:
            # Define a function to update the progress bar after each task completion
            def update(*_):
                pbar.update()
            
            # Submit tasks to the ThreadPoolExecutor
            futures = [executor.submit(process_file, file) for file in files]
            
            # Add the update function to the completion of each future
            for future in concurrent.futures.as_completed(futures):
                future.add_done_callback(update)

folder="./Data/cleaned_text_manually/"
files = glob.glob(f"{folder}*")

# process_files_concurrently(files)







# print(f"Running pipeline for {len(files)} files in /Data/cleaned_text_manually/ folder")

# for i, file in enumerate(files):
#         # print(f"Extracting entities and relationships for {file}")
#         try:
#             with open(file, "r") as f:
#                  text = f.read().rstrip()
#                 #  filename = file.split("/")[-1] 
#                  filename = file.split("/")[-1].replace(".txt", "")  # Extracting just the filename without the path and extension
#                  createGML(text, filename)
#         except Exception as e:
#             print(f"Error processing {file}: {e}")


# # # Create a dictionary to represent the knowledge graph
# graph_data = {
#     "nodes": list(set(triplet[0] for triplet in triples + [triplet[1] for triplet in triples])),
#     "edges": [{"source": triplet[0], "target": triplet[1], "label": triplet[2]} for triplet in triples]
# }

# # Convert graph data to JSON format
# graph_json = json.dumps(graph_data, indent=4)

# # Display or use the JSON data as needed
# print(graph_json)


In [ ]:
folder="./Models/GML/"

#comibine them
# Get all .gml files in the folder
filesGml = glob.glob(f"{folder}*.gml")

# Initialize an empty list to hold graphs
graphs = []
print(f"Combine graphs {filesGml}")
# Read each .gml file and append the graphs to the list
for file in filesGml:
    print(f"file=== {file}")
    G = nx.read_gml(file)
    graphs.append(G)
# Combine all the graphs
combined_graph = nx.compose_all(graphs)
# nx.write_gml(combined_graph, 'combined_graph_model_v.gml')

### Save Graph 


In [20]:
# graph.write_to_gml("Models/GML/AFG-CPD-2014-EN.gml")

### Use Saved Graph


In [46]:
from langchain.indexes.graph import NetworkxEntityGraph
from IPython.display import display, Markdown

loaded_graph = NetworkxEntityGraph.from_gml("moonshot_AI_graph_model_v2.gml")

# prompt =  "Ignore previous output. Use the following knowledge triplets  to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"

prompt =  "Use the following knowledge triplets to answer the question at the end. If you don't know the answer, look out for potential factors in the knowledge triplets else just say I don't know based on my knowledge base, don't try to make up an answer. If a term like a Continent is used e.g Africa, Asia, replace the continent with all african countries available in the knowledge triplets. E.g Nigeria, South Africa and Egypt are under Africa. In your answer, Always refer to knowledge triplets as knowledge base.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"
prompt_entity="Extract all entities from the following text. As a guideline, a proper noun is generally capitalized. You should definitely extract all names,places, Dates and Times, Numbers, Organizations, Products and Brands, Events, Roles and Positions, Keywords and Topics, Email Addresses and URLs, References to External Entities, Emotional Tone, Quantities and Units, Codes and Identifiers, Languages, Social Media Handles, Currencies..\n\nReturn the output as a single comma-separated list, or NONE if there is nothing of note to return.\n\nEXAMPLE\ni'm trying to improve Langchain's interfaces, the UX, its integrations with various products the user might want ... a lot of stuff.\nOutput: Langchain\nEND OF EXAMPLE\n\nEXAMPLE\ni'm trying to improve Langchain's interfaces, the UX, its integrations with various products the user might want ... a lot of stuff. I'm working with Sam.\nOutput: Langchain, Sam\nEND OF EXAMPLE\n\nBegin!\n\n{input}\nOutput:"
question = f"""
What is the most used source of energy in Nigeria?

"""
chain = GraphQAChain.from_llm(AzureChatOpenAI(temperature=0, deployment_name= openai_deployment), graph=loaded_graph, verbose=True,
qa_prompt=PromptTemplate(input_variables=['context', 'question'], template=prompt),
entity_prompt=PromptTemplate(input_variables=['input'], template=prompt_entity)
)
response = chain.run(question) 


# IPython.display.HTML(response)
display(Markdown(response))




> Entering new GraphQAChain chain...
Entities Extracted:
Nigeria
Full Context:
Nigeria is competing for investment
Nigeria lacks critical gas infrastructure
Nigeria continues to fall short of Domestic Gas Supply Obligations
Nigeria is experiencing a full-blown energy crisis
Nigeria has National Gas Policy
Nigeria has Country Name: Nigeria
Nigeria has Country Code: NGA
Nigeria has Category: NGP
Nigeria has Document Title: Nigeria National Gas Policy
Nigeria has Exists?: Y
Nigeria has Publication Date: 2017
Nigeria has Start Year: 2020
Nigeria has End Year: 2030
Nigeria has Language: EN
Nigeria Identify and promote domestic gas market development projects b.
Nigeria Gain more value from international downstream LNG markets c.
Nigeria Clarify gas terms for PSCs d.
Nigeria Achieve gas flare-out through gas utilisation projects utilising mature flare reduction technologies e.
Nigeria Produce a Gas Resource Management Plan f.
Nigeria A review of gas aggregation policy and the future role o

Based on the given knowledge triplets, the most used source of energy in Nigeria is gas. Nigeria is described as a gas play and has proven gas reserves. It is also mentioned that Nigeria has more gas reserves than oil. Additionally, there are various references to Nigeria's gas sector, gas infrastructure, and gas utilization projects. Therefore, it can be inferred that gas is the most used source of energy in Nigeria.